In [1]:
import pandas as pd
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.metrics import mean_squared_error,make_scorer
from imblearn.over_sampling import SMOTE
from xgboost import XGBRegressor

label_col = 'SalePrice'
def preprocess_data(df, encoder=None, scaler=None, fit_encoder=False):
    df = df.copy()

    # Xử lý giá trị thiếu
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].fillna(df[col].mode()[0])
        else:
            df[col] = df[col].fillna(df[col].median())

    # Mã hóa biến phân loại bằng Target Encoding
    categorical_cols = df.select_dtypes(include=["object"]).columns
    if fit_encoder:
        encoder = TargetEncoder()
        df[categorical_cols] = encoder.fit_transform(df[categorical_cols], df[label_col])
    else:
        df[categorical_cols] = encoder.transform(df[categorical_cols])

    # Chuẩn hóa dữ liệu số
    numerical_cols = df.select_dtypes(include=["int64", "float64"]).columns
    numerical_cols = numerical_cols.drop(label_col, errors="ignore")
    if fit_encoder:
        scaler = StandardScaler()
        df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    else:
        df[numerical_cols] = scaler.transform(df[numerical_cols])

    return df, encoder, scaler
df_train.drop(columns=["Id"], errors="ignore", inplace=True)
df_train, encoder, scaler = preprocess_data(df_train, fit_encoder=True)
X_train = df_train.drop(columns=[label_col])
y_train = df_train[label_col]
X_train_resampled, X_val, y_train_resampled, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
param_grid_xgb = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
grid_search_xgb = GridSearchCV(estimator=XGBRegressor(random_state=42), param_grid=param_grid_xgb, cv=3, scoring= rmse_scorer, verbose=2)
grid_search_xgb.fit(X_train_resampled, y_train_resampled)

xgb_model = grid_search_xgb.best_estimator_
print("Best XGBoost parameters:", grid_search_xgb.best_params_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=300, subsample=0.8; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=300, subsample=0.8; total time=   0.5s
[CV] END 

In [8]:
xgb_pred = xgb_model.predict(X_val)
xgb_acc = np.sqrt(mean_squared_error(y_val, xgb_pred))
print("XGBoost Accuracy:", xgb_acc)

XGBoost Accuracy: 24251.101956202307


In [9]:
df_test, _, _ = preprocess_data(df_test, encoder=encoder, scaler=scaler, fit_encoder=False)
xgb_test_pred = xgb_model.predict(df_test)

submission = pd.DataFrame({"Id": df_test["Id"], "SalePrice": xgb_test_pred})
submission.to_csv("submission.csv", index=False)
print("Submission file created!")


KeyError: 'Id'

In [5]:
def preprocess_data(df, encoder=None, scaler=None, fit_encoder=False):
    df = df.copy()

    # Xử lý giá trị thiếu
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].fillna(df[col].mode()[0])
        else:
            df[col] = df[col].fillna(df[col].median())

    # Mã hóa biến phân loại bằng Target Encoding
    categorical_cols = df.select_dtypes(include=["object"]).columns
    if fit_encoder:
        encoder = TargetEncoder()
        df[categorical_cols] = encoder.fit_transform(df[categorical_cols], df[label_col])
    else:
        df[categorical_cols] = encoder.transform(df[categorical_cols])

    # Chuẩn hóa dữ liệu số
    numerical_cols = df.select_dtypes(include=["int64", "float64"]).columns
    numerical_cols = numerical_cols.drop(label_col, errors="ignore")
    if fit_encoder:
        scaler = StandardScaler()
        df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    else:
        df[numerical_cols] = scaler.transform(df[numerical_cols])

    return df, encoder, scaler